In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas as pd
import re
import unicodedata
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm
import pickle
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import matplotlib.pyplot as plt
import itertools
from sklearn.utils import shuffle
!pip -q install keras_bert

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
dftrain=pd.read_csv('/content/drive/MyDrive/scibertfile/trainset_n.csv')
dftest=pd.read_csv('/content/drive/MyDrive/scibertfile/testset.csv')
dfval=pd.read_csv('/content/drive/MyDrive/scibertfile/validset.csv')

In [ ]:
from keras_bert import load_trained_model_from_checkpoint, Tokenizer, get_custom_objects
from keras.layers import *
from keras.layers import Input
from keras.callbacks import *
from keras.models import Model, load_model
import keras.backend as K
from keras.optimizers import Adam
from keras.utils import to_categorical
import os
import keras
SEQ_LEN = 256
config_path = '/content/drive/MyDrive/scibertfile/Scibert/bert_config.json'
checkpoint_path = '/content/drive/MyDrive/scibertfile/Scibert/bert_model.ckpt'
vocab_path = '/content/drive/MyDrive/scibertfile/Scibert/vocab.txt'

token_dict = {}
import codecs, gc, re
with codecs.open(vocab_path, 'r', 'utf8') as reader :
    for line in reader :
        token = line.strip()
        token_dict[token] = len(token_dict)
tokenizer = Tokenizer(token_dict)

In [ ]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
nltk.download('stopwords')
def clean_stopwords_shortwords(w):
    stopwords_list=stopwords.words('english')
    words = w.split() 
    clean_words = [word for word in words if (word not in stopwords_list) and len(word) > 2]
    return " ".join(clean_words) 
def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w=clean_stopwords_shortwords(w)
    w=re.sub(r'@\w+', '',w)
    return w
def label_to_onehot(labels) :
    """ Convert label to onehot .
        Args:
            labels (string): sentence's labels.
        Return:
            outputs (onehot list): sentence's onehot label.
    """
    label_dict = {'THEORETICAL': 0, 'ENGINEERING':1, 'EMPIRICAL':2, 'OTHERS':3}
    onehot = [0,0,0,0]
    for l in labels.split():
        onehot[label_dict[l]] = 1
    return onehot
###Preprocess data###
def process_data(data, test=0) :
    all_x1 = []
    all_x2 = []
    label = []
    for i in range(len(data)) :
        sentid = []
        senti = []
        for sent in data['Abstract'][i]:
            x1, x2 = tokenizer.encode(sent)
            sentid = sentid + x1
            senti = senti + x2
        if test == 0:
            label.append(label_to_onehot(data['Classifications'][i]))
        if len(sentid) > SEQ_LEN :
            all_x1.append(sentid[-int(SEQ_LEN/2):] + sentid[:int(SEQ_LEN/2)])
            all_x2.append(senti[-int(SEQ_LEN/2):]+senti[:int(SEQ_LEN/2)])
        else :
            all_x1.append(sentid + [0] * (SEQ_LEN - len(sentid)))
            all_x2.append(senti + [0] * (SEQ_LEN - len(sentid)))
    if test == 0:
        return [np.asarray(all_x1), np.asarray(all_x2)], label
    else:
        return [np.asarray(all_x1), np.asarray(all_x2)]

In [ ]:
dftrain_c=dftrain.copy()
dftrain_c['Abstract']=dftrain_c['Abstract'].map(preprocess_sentence)
x_train, y_train = process_data(dftrain_c)

dfval_c=dfval.copy()
dfval_c['Abstract']=dfval_c['Abstract'].map(preprocess_sentence)
x_val, y_val = process_data(dfval_c)

dftest_c=dftest.copy()
dftest_c['Abstract']=dftest_c['Abstract'].map(preprocess_sentence)

dftest_c = process_data(dftest_c, 1)

In [ ]:
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
x_val = np.asarray(x_val)
y_val = np.asarray(y_val)
dftest_c = np.asarray(dftest_c)

In [ ]:
class IntervalEvaluation(Callback):
    def __init__(self, validation_data=(), interval=10):
        super(Callback, self).__init__()
        self.interval = interval
        self.X_val, self.y_val = validation_data
    def on_epoch_end(self, epoch, logs={}):
        #if epoch % self.interval == 0:
        y_pred = (np.asarray(self.model.predict([self.X_val[0], self.X_val[1]]))).round().astype(int)
        score = f1_score(self.y_val, y_pred, average='micro')
        print("f1_score - epoch: {:d} - score: {:.6f}".format(epoch+1, score))
get_f1 = IntervalEvaluation(validation_data=(x_val, y_val)) 

In [ ]:
bert_model = load_trained_model_from_checkpoint(
        config_path,
        checkpoint_path,
        training=True,
        trainable=True,
        seq_len=SEQ_LEN,
        )

In [ ]:
keras.backend.clear_session()

inputs = bert_model.inputs[:2]

frontout = bert_model.get_layer('Encoder-4-FeedForward-Norm').output
dense1 = GlobalAveragePooling1D()(frontout)

modelout = bert_model.get_layer('Encoder-12-FeedForward-Norm').output
dense2 = GlobalAveragePooling1D()(modelout)

denseout = keras.layers.Maximum()([dense2, dense1])

denseout = Dropout(0.1)(denseout)

outputs = Dense(4, activation='sigmoid')(denseout)

model = Model(inputs, outputs)

print(model.summary())
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=1e-5), metrics=['binary_accuracy'])
es = EarlyStopping(monitor='val_binary_accuracy', mode='auto', verbose=1, patience=2, restore_best_weights=True)
mc = ModelCheckpoint(filepath='/content/drive/MyDrive/scibertfile/bert_model_3.h5', save_best_only=True)

In [ ]:
history = model.fit([x_train[0], x_train[1]]
                    , y_train, batch_size=4
                    , epochs=50, validation_data=([x_val[0], x_val[1]],y_val)
                    , callbacks=[get_f1,es,mc])

In [ ]:
model = load_model('/content/drive/MyDrive/scibertfile/bert_model_3.h5', custom_objects=get_custom_objects())

In [ ]:
valpred = model.predict([x_val[0], x_val[1]])

In [ ]:
###Function to modify the threshold of sigmoid output###
def mod_round(data, threshold=0.5):
    pred = []
    for i in range(len(data)) :
        pred_line = []
        for j in range(4) :
            if(data[i][j]) > threshold :
                pred_line += [1]
            else :
                pred_line += [0]
        pred.append(pred_line)
    pred = np.asarray(pred).astype(int)
    return pred

###Fix the unreasonable prediction###
def fix_ans(data):
    wrong1 = 0
    wrong2 = 0
    others = 0
    for i in range(len(data)):
        if sum(data[i]) == 0:
            wrong1 += 1
            data[i] = [0,0,0,1]
        if sum(data[i][0:3]) > 0 and data[i][3] == 1:
            wrong2 += 1
            data[i][3] = 0
        if data[i][3] == 1 :
            others += 1
    return data, wrong1, wrong2, others

###Calculate the F1-score when testing with validation data###
def cal_score(data, y_val) :
    TP = 0
    AP = 0
    TL = 0
    for i in range(len(data)) :
        TP = TP + (data[i]*y_val[i]).sum()
        AP = AP + (data[i].sum())
        TL = TL + (y_val[i]).sum()
    precision = TP / AP
    recall = TP / TL
    FS = 2*precision*recall/(precision+recall)
    #print(TP, AP, TL)
    return precision, recall, FS

In [ ]:
pred_v = mod_round(valpred,0)
oprecision, orecall, oFS=cal_score(pred_v, y_val)
j=0
for i in np.arange(0, 0.7, .001):
    precision, recall, FS=cal_score(mod_round(valpred,i), y_val)
    if FS>oFS:
        oFS=FS
        oprecision=precision
        orecall=recall
        j=i
print(j,oprecision, orecall, oFS)

In [ ]:
def SubmitGenerator(prediction, sampleFile, public=True, filename='prediction.csv'):
    """
    Args:
        prediction (numpy array)
        sampleFile (str)
        public (boolean)
        filename (str)
    """
    sample = pd.read_csv(sampleFile)
    submit = {}
    submit['order_id'] = list(sample.order_id.values)
    redundant = len(sample) - prediction.shape[0]
    if public:
        submit['THEORETICAL'] = list(prediction[:,0]) + [0]*redundant
        submit['ENGINEERING'] = list(prediction[:,1]) + [0]*redundant
        submit['EMPIRICAL'] = list(prediction[:,2]) + [0]*redundant
        submit['OTHERS'] = list(prediction[:,3]) + [0]*redundant
    else:
        submit['THEORETICAL'] = [0]*redundant + list(prediction[:,0])
        submit['ENGINEERING'] = [0]*redundant + list(prediction[:,1])
        submit['EMPIRICAL'] = [0]*redundant + list(prediction[:,2])
        submit['OTHERS'] = [0]*redundant + list(prediction[:,3])
    df = pd.DataFrame.from_dict(submit) 
    df.to_csv(filename,index=False)

In [ ]:
CWD = os.getcwd()
SubmitGenerator(pred, 
                os.path.join(CWD,'data/task2_sample_submission.csv'),
                False, 
                os.path.join(CWD,'task2_submission_0102.csv'))